### Install and Import Required Libraries

In [ ]:
## Install Necessary Libraries
!pip install -q youtube-transcript-api langchain-community langchain-google-genai faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [ ]:
## Import Libraries
from youtube_transcript_api import YouTubeTranscriptApi
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI
# from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from google.colab import userdata
import os

In [ ]:
# get the API key from Google Colab Secrets
api_token = userdata.get("GOOGLE_API_KEY")

# set the API Key and print
os.environ['GOOGLE_API_KEY'] = api_token
print(api_token)

Ashutosh


### Step 1: Document Ingestion
##### First we will fetch the transcript of the video using YouTubeTranscriptApi.

In [24]:
## provide the video ID
youtube_video_id = '9EqrUK7ghho'

try:
    ## specifies the language in which video in
    transcript_list = YouTubeTranscriptApi.get_transcript(youtube_video_id, languages=['en'])
    print(transcript_list[0:2])
except Exception as e:
    print(f"Error Occured in Transcription: {e}")

[{'text': '61% of people feel expected to just get', 'start': 0.28, 'duration': 6.639}, {'text': "over stress yeah it's like just power", 'start': 4.759, 'duration': 5.361}]


In [27]:
## combine the list to get the full text
transcript = " ".join([text_dict['text'] for text_dict in transcript_list])
# print(transcript)

In [26]:
## Split the transcript text into small chunks for better search
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
text_chunks = text_splitter.create_documents([transcript])
print(len(text_chunks))

98


In [25]:
text_chunks[0]

Document(metadata={}, page_content="61% of people feel expected to just get over stress yeah it's like just power through the moment CEO of calm author of recharge David code good stress could lead to resilience you're never really going to have a stressfree work environment not everything has to be 24/7 365 because at some point you've caused a lot of bad stress that's a real kind of indicator for you to say I need to really kind of take a hard stop and figure out how I recharge the number one Health and Wellness podcast J shett J shett the one the only J sh hey everyone welcome back to onp purpose the number one place you come to become happier healthier and more healed thank you so much for tuning in today today I'm so excited because I talked to a really good friend someone that I respect so highly and admire so deeply and I've had the fortune of knowing for the last couple of years I'm talking about David Co the CEO of calm the leading app for meditation and mindfulness and a new 

### Embedding and Store
##### Generate the embedding of the text chunks and store them in a vector store for future retrival